1. Imports

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, classification_report
import joblib

print("Librerías cargadas correctamente.")


Librerías cargadas correctamente.


2. Cargar modelo + dataset

In [2]:
def find_project_root(start_path):
    current = start_path
    while True:
        if os.path.exists(os.path.join(current, ".git")):
            return current
        parent = os.path.dirname(current)
        if parent == current:
            raise RuntimeError("No se encontró la raíz del proyecto (.git).")
        current = parent

PROJECT_ROOT = find_project_root(os.getcwd())
DATA_PROCESSED = os.path.join(PROJECT_ROOT, "data", "processed")
MODELS_DIR = os.path.join(PROJECT_ROOT, "models", "sprint4")

DATASET_PATH = os.path.join(DATA_PROCESSED, "dataset_modelado.parquet")

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATASET_PATH:", DATASET_PATH)
print("MODELS_DIR:", MODELS_DIR)


PROJECT_ROOT: c:\IA_Investigacion\DetCorr_Limpio
DATASET_PATH: c:\IA_Investigacion\DetCorr_Limpio\data\processed\dataset_modelado.parquet
MODELS_DIR: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4


In [3]:
df = pd.read_parquet(DATASET_PATH)

# Cargar el mejor modelo
model_files = [f for f in os.listdir(MODELS_DIR) if f.startswith("modelo_final")]
assert len(model_files) > 0, "No hay modelo final en models/sprint4"

best_model_path = os.path.join(MODELS_DIR, model_files[0])
model = joblib.load(best_model_path)

print("Modelo cargado:", best_model_path)

df.head()


Modelo cargado: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4\modelo_final_RandomForest.pkl


,CODIGO_UNICO,SECTOR,DEPARTAMENTO,NIVEL_GOBIERNO,PROCESO,OBJETO_PROCESO,CODIGO_OBRA,METODO_CONTRATACION,TIEMPO_ABSOLUCION_CONSULTAS,TIEMPO_PRESENTACION_OFERTAS,...,MES,PLANIFICADO,REAL,IND_INTERVENSION,IND_RESIDENTE,IND_MONTO_ADELANTO_MATERIALES,IND_MONTO_ADELANTO_DIRECTO,IND_FECHA_ADELANTO_MATERIALES,IND_FECHA_ADELANTO_DIRECTO,riesgo
0,2002060.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,3.0,CONSULTORÍA DE OBRA,19777.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
1,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,60.0,CONSULTORÍA DE OBRA,826.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
2,2002210.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,60.0,CONSULTORÍA DE OBRA,826.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
3,2015918.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,24.0,CONSULTORÍA DE OBRA,45660.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0
4,2026767.0,TRANSPORTE,MULTIDEPARTAMENTAL,GOBIERNO NACIONAL,14.0,CONSULTORÍA DE OBRA,143536.0,CONCURSO PÚBLICO,0.0,0.0,...,4.0,0.185,0.1,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,DESCONOCIDO,-1,-1,0.0


3. Definir columnas X e y para evaluación

In [4]:
y = df["riesgo"]
X = df.drop(columns=["riesgo"])

categoricas = X.select_dtypes(include=["object"]).columns.tolist()
numericas = X.select_dtypes(exclude=["object"]).columns.tolist()

print("Categóricas:", categoricas)
print("Numéricas:", numericas)


Categóricas: ['CODIGO_UNICO', 'SECTOR', 'DEPARTAMENTO', 'NIVEL_GOBIERNO', 'OBJETO_PROCESO', 'METODO_CONTRATACION', 'ESTADO_OBRA', 'ETAPA', 'IND_INTERVENSION', 'IND_RESIDENTE', 'IND_MONTO_ADELANTO_MATERIALES', 'IND_MONTO_ADELANTO_DIRECTO']
Numéricas: ['PROCESO', 'CODIGO_OBRA', 'TIEMPO_ABSOLUCION_CONSULTAS', 'TIEMPO_PRESENTACION_OFERTAS', 'CODIGO_RUC', 'CONVOCATORIA_PROCESO_GANADO', 'TOTALPROCESOSPARTICIPANTES', 'CODIGO_CONTRATO', 'MONTO_CONTRACTUAL', 'MONTO_REFERENCIAL', 'MONTO_OFERTADO_PROMEDIO', 'CONVOCATORIA', 'DNI_MIEMBRO_COMITE', 'CODIGO_RUC_GANADOR', 'CODIGO_RUC_PARTICIPANTE', 'RUC_GANADOR', 'RUC_PARTICIPANTE', 'MONTO_OFERTADO', 'DIAS_PLAZO', 'TOTAL_CONTROL_PREVIO', 'TOTAL_CONTROL_SIMULTANEO', 'TOTAL_CONTROL_POSTERIOR', 'RIESGO_OBRA', 'ANHO', 'MES', 'PLANIFICADO', 'REAL', 'IND_FECHA_ADELANTO_MATERIALES', 'IND_FECHA_ADELANTO_DIRECTO']


4. Evaluación global (baseline)

In [5]:
y_pred = model.predict(X)
y_prob = model.predict_proba(X)[:, 1]

baseline_results = {
    "f1": f1_score(y, y_pred),
    "accuracy": accuracy_score(y, y_pred),
    "roc_auc": roc_auc_score(y, y_prob)
}

baseline_results


{'f1': 0.8560867422375554,
 'accuracy': 0.9176246561816771,
 'roc_auc': 0.9267481575721985}

5. Definir slices a evaluar

In [6]:
slices = [
    "SECTOR",
    "DEPARTAMENTO",
    "NIVEL_GOBIERNO",
    "OBJETO_PROCESO",
    "METODO_CONTRATACION",
    "ESTADO_OBRA",
    "ETAPA"
]

slices = [s for s in slices if s in df.columns]

print("Slices a evaluar:", slices)



Slices a evaluar: ['SECTOR', 'DEPARTAMENTO', 'NIVEL_GOBIERNO', 'OBJETO_PROCESO', 'METODO_CONTRATACION', 'ESTADO_OBRA', 'ETAPA']


6. Función para evaluar cada slice

In [7]:
def evaluar_slice(df, columna, modelo):
    resultados = []
    valores = df[columna].unique()
    
    for val in valores:
        df_slice = df[df[columna] == val]
        
        if len(df_slice) < 30:
            # evitar segmentos muy pequeños
            continue
        
        y_true = df_slice["riesgo"]
        X_slice = df_slice.drop(columns=["riesgo"])
        
        y_pred = modelo.predict(X_slice)
        y_prob = modelo.predict_proba(X_slice)[:, 1]
        
        resultados.append({
            "slice_columna": columna,
            "slice_valor": val,
            "n": len(df_slice),
            "f1": f1_score(y_true, y_pred),
            "accuracy": accuracy_score(y_true, y_pred),
            "roc_auc": roc_auc_score(y_true, y_prob),
        })
    
    return resultados


##### Evaluación con Intervalos de Confianza (IC)

In [8]:
def evaluar_slice_con_ic(model, X_slice, y_slice, B=200):
    """Evalúa un slice con métricas (F1, Acc, AUC) + intervalos de confianza Bootstrap."""
    try:
        # Predicciones base
        y_pred = model.predict(X_slice)

        # Si no existe predict_proba, usar decision_function o marcarlo NaN
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_slice)[:, 1]
        elif hasattr(model, "decision_function"):
            scores = model.decision_function(X_slice)
            # Normalizar a [0,1]
            y_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)
        else:
            y_proba = np.zeros(len(y_slice))  # fallback seguro

        # Métricas base
        f1 = f1_score(y_slice, y_pred)
        acc = accuracy_score(y_slice, y_pred)
        try:
            auc = roc_auc_score(y_slice, y_proba)
        except:
            auc = np.nan

        # --- BOOTSTRAP ---
        f1_bs, acc_bs, auc_bs = [], [], []
        n = len(y_slice)

        for _ in range(B):
            idx = np.random.choice(n, n, replace=True)
            y_b = y_slice.iloc[idx]
            X_b = X_slice.iloc[idx]

            y_pred_b = model.predict(X_b)

            if hasattr(model, "predict_proba"):
                y_proba_b = model.predict_proba(X_b)[:, 1]
            elif hasattr(model, "decision_function"):
                scores_b = model.decision_function(X_b)
                y_proba_b = (scores_b - scores_b.min()) / (scores_b.max() - scores_b.min() + 1e-9)
            else:
                y_proba_b = np.zeros(len(y_b))

            f1_bs.append(f1_score(y_b, y_pred_b))
            acc_bs.append(accuracy_score(y_b, y_pred_b))
            try:
                auc_bs.append(roc_auc_score(y_b, y_proba_b))
            except:
                auc_bs.append(np.nan)

        # IC del 95%
        f1_ic = (np.nanpercentile(f1_bs, 2.5), np.nanpercentile(f1_bs, 97.5))
        acc_ic = (np.nanpercentile(acc_bs, 2.5), np.nanpercentile(acc_bs, 97.5))
        auc_ic = (np.nanpercentile(auc_bs, 2.5), np.nanpercentile(auc_bs, 97.5))

        return {
            "f1": f1, "f1_ic_low": f1_ic[0], "f1_ic_high": f1_ic[1],
            "accuracy": acc, "acc_ic_low": acc_ic[0], "acc_ic_high": acc_ic[1],
            "roc_auc": auc, "auc_ic_low": auc_ic[0], "auc_ic_high": auc_ic[1]
        }

    except Exception as e:
        print("Error en slice:", e)
        return {
            "f1": np.nan, "f1_ic_low": np.nan, "f1_ic_high": np.nan,
            "accuracy": np.nan, "acc_ic_low": np.nan, "acc_ic_high": np.nan,
            "roc_auc": np.nan, "auc_ic_low": np.nan, "auc_ic_high": np.nan
        }


7. Ejecutar la evaluación de slices

In [9]:
all_results = []

for col in slices:
    print("\nEvaluando slice:", col)
    res = evaluar_slice(df, col, model)
    all_results.extend(res)

df_slices = pd.DataFrame(all_results)
df_slices




Evaluando slice: SECTOR

Evaluando slice: DEPARTAMENTO

Evaluando slice: NIVEL_GOBIERNO

Evaluando slice: OBJETO_PROCESO

Evaluando slice: METODO_CONTRATACION

Evaluando slice: ESTADO_OBRA

Evaluando slice: ETAPA


,slice_columna,slice_valor,n,f1,accuracy,roc_auc
0,SECTOR,TRANSPORTE,3086,0.846765,0.912508,0.919192
1,SECTOR,AGRARIA,107,0.852459,0.915888,0.956388
2,SECTOR,INDUSTRIA; COMERCIO Y SERVICIOS,79,0.833333,0.898734,0.911483
3,SECTOR,SALUD Y SANEAMIENTO,84,0.898551,0.916667,0.974450
4,SECTOR,OTROS,1311,0.840696,0.909230,0.923589
...,...,...,...,...,...,...
59,ETAPA,DESCONOCIDO,13853,0.856388,0.917635,0.926378
60,ETAPA,ETAPA NO IDENTIFICADA,85,0.820513,0.917647,0.905483
61,ETAPA,EXPEDIENTE TÉCNICO,69,0.823529,0.913043,0.980612
62,ETAPA,SUPERVISIÓN,99,0.836364,0.909091,0.903202


##### Con intervalos de Confianza (IC)

In [10]:
resultados_slices_ic = []

for col in slices:
    valores_unicos = df[col].unique()

    for val in valores_unicos:
        df_sub = df[df[col] == val]

        # Evitar slices muy pequeños
        if len(df_sub) < 30:
            continue

        X_slice = df_sub.drop(columns=["riesgo"])
        y_slice = df_sub["riesgo"]

        met = evaluar_slice_con_ic(model, X_slice, y_slice)

        resultados_slices_ic.append({
            "slice_columna": col,
            "slice_valor": val,
            "n": len(df_sub),
            **met
        })

df_slices_ic = pd.DataFrame(resultados_slices_ic)
df_slices_ic.head()


c:\IA_Investigacion\DetCorr_Limpio\env\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


,slice_columna,slice_valor,n,f1,f1_ic_low,f1_ic_high,accuracy,acc_ic_low,acc_ic_high,roc_auc,auc_ic_low,auc_ic_high
0,SECTOR,TRANSPORTE,3086,0.846765,0.829987,0.865106,0.912508,0.903427,0.921589,0.919192,0.907163,0.931726
1,SECTOR,AGRARIA,107,0.852459,0.753996,0.948810,0.915888,0.859813,0.971963,0.956388,0.902400,0.990290
2,SECTOR,INDUSTRIA; COMERCIO Y SERVICIOS,79,0.833333,0.693718,0.933469,0.898734,0.835127,0.962025,0.911483,0.834134,0.979065
3,SECTOR,SALUD Y SANEAMIENTO,84,0.898551,0.807003,0.955580,0.916667,0.857143,0.964286,0.974450,0.937150,0.996981
4,SECTOR,OTROS,1311,0.840696,0.804916,0.866348,0.909230,0.890141,0.924523,0.923589,0.904205,0.945241


In [ ]:
#####OPTIMIZADO#####
resultados_slices_ic = []

# Reducir análisis a slices relevantes
slices = [
    "SECTOR",
    "DEPARTAMENTO",
    "NIVEL_GOBIERNO",
    "METODO_CONTRATACION",
    "ESTADO_OBRA",
    "ETAPA"
]

for col in slices:
    print(f"Analizando columna: {col}")
    valores_unicos = df[col].unique()

    for val in valores_unicos:
        df_sub = df[df[col] == val]

        # Evitar slices pequeños o sin variabilidad
        if len(df_sub) < 100:
            continue
        if len(df_sub["riesgo"].unique()) < 2:
            continue

        X_slice = df_sub.drop(columns=["riesgo"])
        y_slice = df_sub["riesgo"]

        met = evaluar_slice_con_ic(model, X_slice, y_slice)

        resultados_slices_ic.append({
            "slice_columna": col,
            "slice_valor": val,
            "n": len(df_sub),
            **met
        })

df_slices_ic = pd.DataFrame(resultados_slices_ic)
df_slices_ic.head()


8. Ordenar y detectar problemas

In [11]:
df_slices_sorted = df_slices.sort_values(by="f1")
df_slices_sorted.head(20)


,slice_columna,slice_valor,n,f1,accuracy,roc_auc
7,SECTOR,DEFENSA Y SEGURIDAD NACIONAL,50,0.571429,0.880000,0.667500
9,SECTOR,PESCA,30,0.625000,0.800000,0.925466
32,DEPARTAMENTO,ICA,33,0.736842,0.848485,0.801653
14,SECTOR,JUSTICIA,49,0.774194,0.857143,0.852941
57,ESTADO_OBRA,EN EJECUCIÓN,133,0.781250,0.894737,0.927405
31,DEPARTAMENTO,AYACUCHO,89,0.782609,0.887640,0.874375
26,DEPARTAMENTO,LA LIBERTAD,53,0.800000,0.905660,0.846154
12,SECTOR,PLANEAMIENTO; GESTIÓN Y RESERVA DE CONTINGENCIA,123,0.800000,0.926829,0.887986
34,DEPARTAMENTO,APURIMAC,33,0.800000,0.909091,0.857500
30,DEPARTAMENTO,PIURA,140,0.810127,0.892857,0.921902


In [12]:
df_slices_sorted_ic = df_slices_ic.sort_values(by="f1")
df_slices_sorted_ic.head(20)

,slice_columna,slice_valor,n,f1,f1_ic_low,f1_ic_high,accuracy,acc_ic_low,acc_ic_high,roc_auc,auc_ic_low,auc_ic_high
7,SECTOR,DEFENSA Y SEGURIDAD NACIONAL,50,0.571429,0.200000,0.800000,0.880000,0.780000,0.960000,0.667500,0.418158,0.913623
9,SECTOR,PESCA,30,0.625000,0.222222,0.857589,0.800000,0.700000,0.933333,0.925466,0.777778,1.000000
32,DEPARTAMENTO,ICA,33,0.736842,0.461111,0.923333,0.848485,0.726515,0.969697,0.801653,0.589173,0.962726
14,SECTOR,JUSTICIA,49,0.774194,0.580415,0.909091,0.857143,0.754592,0.939286,0.852941,0.730274,0.961476
57,ESTADO_OBRA,EN EJECUCIÓN,133,0.781250,0.655157,0.884016,0.894737,0.834586,0.940038,0.927405,0.846547,0.982217
31,DEPARTAMENTO,AYACUCHO,89,0.782609,0.631579,0.892321,0.887640,0.820225,0.943820,0.874375,0.760744,0.965112
26,DEPARTAMENTO,LA LIBERTAD,53,0.800000,0.588235,0.941331,0.905660,0.811321,0.981132,0.846154,0.676792,0.989133
12,SECTOR,PLANEAMIENTO; GESTIÓN Y RESERVA DE CONTINGENCIA,123,0.800000,0.631476,0.920256,0.926829,0.878049,0.975610,0.887986,0.783061,0.964604
34,DEPARTAMENTO,APURIMAC,33,0.800000,0.500000,1.000000,0.909091,0.818182,1.000000,0.857500,0.629467,1.000000
30,DEPARTAMENTO,PIURA,140,0.810127,0.727273,0.898010,0.892857,0.849821,0.943036,0.921902,0.859070,0.973491


9. Guardar resultados para el Sprint

In [13]:
output_path = os.path.join(MODELS_DIR, "slices_problematicos.csv")
df_slices_sorted.to_csv(output_path, index=False)

print("Resultados guardados en:", output_path)


Resultados guardados en: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4\slices_problematicos.csv


In [14]:
output_path_ic = os.path.join(MODELS_DIR, "slices_problematicos_ic.csv")
df_slices_ic.to_csv(output_path_ic, index=False)
print("Slices con intervalos de confianza guardados en:", output_path_ic)


Slices con intervalos de confianza guardados en: c:\IA_Investigacion\DetCorr_Limpio\models\sprint4\slices_problematicos_ic.csv


10. Reporte del peor slice

In [15]:
peor = df_slices_sorted.iloc[0]
print("Slice más problemático:")
peor


Slice más problemático:


slice_columna                          SECTOR
slice_valor      DEFENSA Y SEGURIDAD NACIONAL
n                                          50
f1                                   0.571429
accuracy                                 0.88
roc_auc                                0.6675
Name: 7, dtype: object

In [16]:
peor = df_slices_ic.iloc[0]
print("Slice más problemático con IC:")
peor


Slice más problemático con IC:


slice_columna        SECTOR
slice_valor      TRANSPORTE
n                      3086
f1                 0.846765
f1_ic_low          0.829987
f1_ic_high         0.865106
accuracy           0.912508
acc_ic_low         0.903427
acc_ic_high        0.921589
roc_auc            0.919192
auc_ic_low         0.907163
auc_ic_high        0.931726
Name: 0, dtype: object